# Анализ клинических данных пациентов и хроматограмм

Это задание выполнено на языке **R** в среде **Google Colab**.

**Что сделано:**
1. Загрузка и чтение данных
2. Анонимизация персональной информации
3. Предположение о типе исследования
4. Анализ переменных и их шкал
5. Предположение эмпирических гипотез


In [1]:
%load_ext rpy2.ipython

In [5]:
%%R
install.packages("readr", repos='http://cran.us.r-project.org')
install.packages("dplyr", repos='http://cran.us.r-project.org')
install.packages("ggplot2", repos='http://cran.us.r-project.org')
install.packages("tidyr", repos='http://cran.us.r-project.org')

library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/readr_2.1.5.tar.gz'
Content type 'application/x-gzip' length 298065 bytes (291 KB)
downloaded 291 KB


The downloaded source packages are in
	‘/tmp/RtmpMHW3yg/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/dplyr_1.1.4.tar.gz'
Content type 'application/x-gzip' length 1207521 bytes (1.2 MB)
downloaded 1.2 MB


The downloaded source packages are in
	‘/tmp/RtmpMHW3yg/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'http://cran.us.r-project.org/src/contrib/ggplot2_3.5.2.tar.gz'
Content type 'application/x-gzip' length 3580451 bytes (3.4 MB)
downloaded 3.4 MB


The downloaded source packages are in
	‘/tmp/RtmpMHW3yg/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-libr

In [12]:
%%R
# Загрузка и распаковка ZIP-файла
unzip("test_data.zip", exdir = "test_data")
files<-list.files("test_data", full.names=TRUE)
# Чтение всех TSV-файлов в список
data_list<-lapply(files, read_tsv)
# Просмотр первых строк каждого файла
lapply(data_list, head)


Rows: 27 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): Name
dbl (37): Vial.number, Patient.number, GPHDL1, GPHDL2, GPHDL3, GPHDL4, GPHDL...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 29 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): Name
dbl (37): Vial.number, Patient.number, GPLDL1, GPLDL2, GPLDL3, GPLDL4, GPLDL...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 38 Columns: 49
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr   (2): Name, comorbidities
dbl  (46): Vialnumber, Patientnumber, Sucrose, Group, Age, Plasmavolume, Ht,...
date  (1): Date

ℹ Use `spec()` to 

In [13]:
%%R
# Функция анонимизации данных: замена ID и имён
anonymize <- function(df) {
  df <- df %>%
    mutate(across(contains("name"), ~"anon_name")) %>%
    mutate(across(contains("id"), ~sample(10000:99999, n(), replace = FALSE)))
  return(df)
}

# Применение анонимизации ко всем таблицам
data_anon <- lapply(data_list, anonymize)
lapply(data_anon, head)

[[1]]
# A tibble: 6 × 38
  Vial.number Patient.number Name      GPHDL1 GPHDL2 GPHDL3 GPHDL4 GPHDL5 GPHDL6
        <dbl>          <dbl> <chr>      <dbl>  <dbl>  <dbl>  <dbl>  <dbl>  <dbl>
1           1             50 anon_name   0.38   1.78   0.14   0.05   0.73   2.9 
2           3             52 anon_name   0.43   1.3    0.23   0.05   0.59   2.62
3           4             53 anon_name   0.41   3.86   0.31   0.1    0.96   3.19
4           5             54 anon_name   0.5    1.79   0.23   0.06   0.77   2.96
5           6             55 anon_name   0.4    3.69   0.17   0.03   0.8    3.04
6           7             56 anon_name   0.37   2.96   0.22   0.05   1      3   
# ℹ 29 more variables: GPHDL7 <dbl>, GPHDL8 <dbl>, GPHDL9 <dbl>, GPHDL10 <dbl>,
#   GPHDL11 <dbl>, GPHDL12 <dbl>, GPHDL13 <dbl>, GPHDL14 <dbl>, GPHDL15 <dbl>,
#   GPHDL16 <dbl>, GPHDL17 <dbl>, GPHDL18 <dbl>, GPHDL19 <dbl>, GPHDL20 <dbl>,
#   GPHDL21 <dbl>, GPHDL22 <dbl>, sum_auc <dbl>, LB <dbl>, HB <dbl>, G1 <dbl>,
#   G2 <db

# 3. Тип исследования.
На основе данных, можно предположить, что проводится:

 Наблюдательное исследование(обсервационное)
Пациентам не давали конкретное лечение, а просто измеряли показатели: холестерин, сахар, давление и т.д.

У них брали анализы крови и другие данные, и потом сравнивали группы: например, по возрасту, болезням и т.п.

Также возможно, что изучается влияние уровня ЛПВП и ЛПНП на здоровье.

# 4. Тип выборки и переменные

Тип выборки:
Похоже, это взрослые пациенты с возможными проблемами с сердцем, сосудами, обменом веществ (например, гипертония, диабет, инфаркт).

Все пациенты имеют данные крови и результаты хроматографии (GPHDL и GPLDL — это анализы липидов).

# Переменные и их тип:
| Переменная    | Что означает                                 | Тип значения           | Шкала измерения              |
|---------------|-----------------------------------------------|-------------------------|------------------------------|
| Age           | Возраст пациента                              | Число                   | Шкала отношений              |
| Name          | Имя пациента                                  | Текст                   | Номинальная                  |
| HDLi, LDLi    | Уровни холестерина                            | Число                   | Шкала отношений              |
| Group         | Группа пациента (0 или 1)                     | Метка (контроль / больные) | Номинальная              |
| HT, MI, Str   | Есть ли болезнь (гипертония, инфаркт, инсульт) | 0 или 1 (нет/да)       | Номинальная                  |
| sum_auc       | Сумма под кривой хроматограммы               | Число                   | Шкала отношений              |
| BMI           | Индекс массы тела                             | Число                   | Шкала отношений              |


# 5. Эмпирические гипотезы (что можно проверить)


**Гипотеза 1:**
У пациентов с гипертонией (HT = 1) выше уровень холестерина LDLi, чем у тех, у кого гипертонии нет (HT = 0).

Почему: гипертония часто идёт вместе с высоким "плохим" холестерином.

**Гипотеза 2:**
Уровень HDLi (полезного холестерина) снижается с возрастом.

Проверяется с помощью корреляции: как Age связан с HDLi.

**Гипотеза 3:**
Пациенты, у которых был инфаркт (MI = 1), имеют ниже sum_auc по ЛПВП-хроматограмме, чем те, у кого не было инфаркта.

Возможно, у них меньше защитных фракций HDL.
